In [2]:
# Import packages in alphabetical order
import pandas as pd
import numpy as np
import tiktoken
import os
import openai
import duckdb
import owlready2
import sklearn

import gensim
import matplotlib.pyplot as plt
import nltk
import pickle
import re
import spacy
import string

# Import packages in alphabetical order to avoid duplicates
#from gensim.models import Word2Vec
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
#from nltk.tokenize import word_tokenize
#from sklearn.cluster import KMeans
#from transformers import BertModel, BertTokenizer
from wordcloud import WordCloud

#Import libraries
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
pip install --upgrade accelerate


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
#Load the Mistral Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("quantumaikr/quantum-dpo-v0.1")
model = AutoModelForCausalLM.from_pretrained("quantumaikr/quantum-dpo-v0.1", torch_dtype=torch.float16, device_map="auto")

system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."

message = "Write me a poem please"
prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=2048)

print(tokenizer.decode(output[0], skip_special_tokens=True))


2023-12-20 13:58:45.441754: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 13:58:45.441807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 13:58:45.441846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 13:58:45.451732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.
<</SYS>>

Write me a poem please[/INST] Here /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/**/ /******/ /******/ /******/ /******/ /******/ /******/ /******/ /******/**/ /******/ /******/ /******/ /******/**/

Stars twinkle above,
In the vast, endless sky;
Echoes of silence,
Whispers of beauty, they lie.

Leaves rustle, a gentle breeze,
Swaying in harmony;
Nature's symphony,
A serenade for all to see.

As the sun sets, its golden glow,
Fades into the night;
Dreams and memories,
A gentle lullaby, a soft light.

We dance through life, as one,
In perfect synchronicity;
The rhythm of the universe,
A divine melody, a mystery.

For in the grandest of schemes,
Lies the beauty of existence;
In the smallest of moments,
We find our true essence.


In [3]:
with open('Datasets/grouped_sentences.pkl', 'rb') as f:
    grouped_sentences = pickle.load(f)

In [7]:
#Call group_sentences head
grouped_sentences.head()

,article_id,sentence,processed_sentence
0,1,Examining interdependence between product user...,examining interdependence between product user...
1,2,Information management as an enabler of knowle...,information management as an enabler of knowle...
2,3,Action Research and Critical Rationalisationis...,action research and critical rationalisationis...
3,4,Integration of Supply Chain Execution in B2B M...,integration of supply chain execution in marke...
4,5,Association for Information Systems AIS Electr...,association for information systems ais electr...


In [15]:
#Create sample text
sample_text = ["A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation. LLMs are artificial neural networks (mainly transformers) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word. Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results. They are thought to acquire knowledge about syntax, semantics and ontology inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5] Notable examples include OpenAI's GPT models (e.g., GPT-3.5 and GPT-4, used in ChatGPT), Google's PaLM (used in Bard), and Meta's LLaMA, as well as BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2."] #"A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5] Notable examples include OpenAI's GPT models (e.g., GPT-3.5 and GPT-4, used in ChatGPT), Google's PaLM (used in Bard), and Meta's LLaMA, as well as BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2."

In [16]:
#Test the model with sample text of 180 words
system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."

message = "Please summarise the following text: " + sample_text[0]
prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=2048)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.
<</SYS>>

Please summarise the following text: A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation. LLMs are artificial neural networks (mainly transformers) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word. Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar

In [31]:
#Test the model with sample text of 180 words
system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."

message = "Please summarise the following text: " + sample_text[0]
prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=2048)

#Print result and trim everything before and after the summary
print(tokenizer.decode(output[0], skip_special_tokens=True).split("[/INST]")[1].split("<</SYS>>")[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Large language models (LLMs) are advanced AI systems capable of understanding and generating human-like language. They achieve this through extensive training on massive datasets, learning billions of parameters, and using significant computational resources. LLMs are artificial neural networks, primarily transformers, trained with self- or semi-supervised learning. These models work by taking an input text and progressively predicting the next token or word.

Before 2020, models required fine-tuning to adapt them for specific tasks, but larger models like GPT-3 can be prompt-engineered to accomplish similar results. LLMs are believed to absorb knowledge of syntax, semantics, and ontology from human language corpora but may also learn inaccuracies and biases present in these datasets. Notable examples include OpenAI's GPT models, Google's PaLM, Meta's LLaMA, BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2.


This proves that the model is working, now trying with one row of data from the dataframe

In [17]:
#Test the model with one row
system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."

message = "Please summarise the following text: " + grouped_sentences['sentence'][0]
prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=2048)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 GiB. GPU 0 has a total capacty of 31.74 GiB of which 4.73 GiB is free. Process 408359 has 10.96 GiB memory in use. Process 464204 has 4.18 GiB memory in use. Including non-PyTorch memory, this process has 11.87 GiB memory in use. Of the allocated memory 7.93 GiB is allocated by PyTorch, and 3.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

The research text seems to be so long even with only one row. Trying to process with for loops and chunks

In [20]:
#Combining for loop for chunking and summarising
def generate_summary(article_text, max_chunk_size=1024, max_summary_length=100, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        
        system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."
        message = "Please summarise the following text: " + chunk

        prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate the summary
        output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=2048)

        # Decode the summary
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summary_parts.append(summary)

    return " ".join(summary_parts)

In [21]:
#Trying chunks with one row
article_text = grouped_sentences['sentence'][0]

# Generate summary for the entire article
final_summary = generate_summary(article_text)

# Print the generated summary
print("Original Article:")
print(article_text)
print("\nGenerated Summary:")
print(final_summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Original Article:
Examining interdependence between product users users and employees in online user communities : The role of employee - generated content Abstract. Firm - sponsored online user communities have become product innovation and support hubs of strategic importance to firms . Product users and host firm employees comprise the participants of firm - sponsored online user communities . The online user community provides a forum wherein the product users users and firm employees discuss questions , problems or issues resulting from the use of host firms ' products . Extant research on online user communities has largely focused on either product users users or employees and has examined the various dynamics that ensue from each entity 's community participation . This paper seeks to investigate the interdependence between the two entities in the communities and , in particular , how product users users ' reading of employee - generated content influences subsequent knowledge 

It can be done, but the summarization is too fragmented. Increasing max chunk size and reducing summarization size.

In [14]:
#Combining for loop for chunking and summarising
def gen_summary(article_text, max_chunk_size=16384, max_summary_length=256, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        
        system_prompt = "You are QuantumLM, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal."
        message = "Please summarise the following text: " + chunk

        prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message}[/INST]"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate the summary
        output = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=256)

        # Decode the summary
        summary = tokenizer.decode(output[0], skip_special_tokens=True).split("[/INST]")[1].split("<</SYS>>")[0]
        summary_parts.append(summary)

    return " ".join(summary_parts)

In [9]:
#Trying chunks with one row
article_text = grouped_sentences['processed_sentence'][9999]

# Generate summary for the entire article
final_summary = gen_summary(article_text)

# Print the generated summary
print("\nGenerated Summary:")
print(final_summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Summary:
 The Paper focuses on an Agent Mediated Knowledge in Motion Model within the context of millions invested in Knowledge Management. Researchers in this field explore how firms can organize their processes to reap benefits, yet most research is fragmented and over-emphasizes knowledge creation, with little attention to key intermediaries in the process.

The proposed Agent Mediated Knowledge in Motion Model embodies a cycle of knowledge creation and reuse, tying agent-based research to knowledge creation and application. Organizations can strategically employ human and software agents to enhance the creation, transfer, application, and dissemination of knowledge in the process. The paper highlights specific roles and attributes of various agents in the knowledge management process, using the organization as the primary unit of analysis.

The scope of discussion revolves around the conceptualization of an Agent Mediated Knowledge Management process, where data is trans

The chunking with the manual fine tuning of the model looks good enough. Now continuing to random sampling the dataset for further analysis.

In [36]:
#Pick 100 rows with random state 42 from the grouped_sentences dataframe
grouped_sentences_100 = grouped_sentences.sample(n=100, random_state=42)

In [37]:
#Save grouped_sentences_random to pickle
with open('Datasets/grouped_sentences_100.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_100, f)

In [26]:
#Call group_sentences_100 head
grouped_sentences_100.head()

,article_id,sentence,processed_sentence
50374,50450,Association for Information Systems AIS Electr...,association for information systems ais electr...
13657,13667,Communications of the Association for Informat...,communications of the association for informat...
11449,11457,Effective organizational improvisation in info...,effective organizational improvisation in info...
46481,46549,Researching the Costs of Information Systems T...,researching the costs of information systems t...
30546,30587,Holographic Recommendations in Brick - and - M...,holographic recommendations in brick and morta...


In [ ]:
#Call group_sentences_100 first row column processed_sentence
grouped_sentences_100['processed_sentence'][0]

In [28]:
#Create a copy of the grouped_sentences_100 dataframe as grouped_sentences_100_m
grouped_sentences_100_m = grouped_sentences_100.copy()

In [29]:
#Create a for loop for every row of grouped_sentences_100_m column 'processed_sentence' as article_text and save the result in a new column 'summary'
for index, row in grouped_sentences_100_m.iterrows():
    article_text = row['processed_sentence']
    final_summary = gen_summary(article_text)
    grouped_sentences_100_m.loc[index, 'summary'] = final_summary

#Save grouped_sentences_100 to pickle
with open('Datasets/grouped_sentences_100_m.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_100_m, f)

#Call group_sentences_100_m head
grouped_sentences_100_m.head()
    

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

,article_id,sentence,processed_sentence
50374,50450,Association for Information Systems AIS Electr...,association for information systems ais electr...
13657,13667,Communications of the Association for Informat...,communications of the association for informat...
11449,11457,Effective organizational improvisation in info...,effective organizational improvisation in info...
46481,46549,Researching the Costs of Information Systems T...,researching the costs of information systems t...
30546,30587,Holographic Recommendations in Brick - and - M...,holographic recommendations in brick and morta...


In [31]:
#Set grouped_sentences_100 as a new variable grouped_sentences_100_mistral
grouped_sentences_100_mistral = grouped_sentences_100.copy()

In [33]:
#Call grouped_sentences_100_mistral head
grouped_sentences_100_mistral.head()

,article_id,sentence,processed_sentence,summary
50374,50450,Association for Information Systems AIS Electr...,association for information systems ais electr...,The study aims to examine the differences in ...
13657,13667,Communications of the Association for Informat...,communications of the association for informat...,The longitudinal study on the supply and dema...
11449,11457,Effective organizational improvisation in info...,effective organizational improvisation in info...,The study focuses on understanding organizati...
46481,46549,Researching the Costs of Information Systems T...,researching the costs of information systems t...,The paper examines research gaps in understan...
30546,30587,Holographic Recommendations in Brick - and - M...,holographic recommendations in brick and morta...,The researchers propose an in-store recommend...


In [ ]:
#Save grouped_sentences_100_mistral to pickle
with open('Datasets/grouped_sentences_100_mistral.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_100_mistral, f)

In [7]:
#Import grouped_sentences_100 from pickle
with open('Datasets/grouped_sentences_100.pkl', 'rb') as f:
    grouped_sentences_100 = pickle.load(f)

In [8]:
#Provide me grouped_sentences after taking out the grouped_sentences_100
grouped_sentences_min100 = grouped_sentences.drop(grouped_sentences_100.index)

In [9]:
#Pick 300 rows with random state 42 from the grouped_sentences_min100 dataframe
grouped_sentences_300 = grouped_sentences_min100.sample(n=300, random_state=42)

In [11]:
#call grouped_sentences_300 head
grouped_sentences_300.head()

,article_id,sentence,processed_sentence
17899,17917,The firm 's continuance intentions to use inte...,the firm continuance intentions to use inter o...
9564,9570,The Philosopher 's Corner : Beyond Epistemolog...,the philosopher corner beyond epistemology and...
45598,45666,Interplay of Competition where ln Innovation w...,interplay of competition where ln innovation w...
39260,39308,The Impact of Crowdsourcing on Organisational ...,the impact of crowdsourcing on organisational ...
23407,23440,Association for Information Systems Associatio...,association for information systems associatio...


In [12]:
#Save grouped_sentences_300 to pickle
with open('Datasets/grouped_sentences_300.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_300, f)

In [13]:
#Set grouped_sentences_300 as a new variable grouped_sentences_300_m
grouped_sentences_300_m = grouped_sentences_300.copy()

In [19]:
#Create a for loop for every row of grouped_sentences_300_m column 'processed_sentence' as article_text and save the result in a new column 'summary_mistral'
for index, row in grouped_sentences_300_m.iterrows():
    article_text = row['processed_sentence']
    final_summary = gen_summary(article_text)
    grouped_sentences_300_m.loc[index, 'summary_mistral'] = final_summary

#Save grouped_sentences_300 to pickle
with open('Datasets/grouped_sentences_300_m.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_300_m, f)

#Call group_sentences_300_m head
grouped_sentences_300_m.head()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
